In [ ]:
import pandas as pd
import numpy as np
# !pip install transformers
# from transformers import pipeline
# # !pip install bert-embedding
# import torch
# import torch.nn.functional as F
import matplotlib.pyplot as plt
import pickle

In [ ]:
df = pd.read_csv('cleaned-dataset.tsv', sep = '\t')
df_dixon = pd.read_csv('/content/pred_output_dixcon_coles.tsv', sep = '\t')
df_parse = pd.read_csv('/content/parse_tree_clause_allocations_aggregated.tsv', sep = '\t')
df_openie = pd.read_csv('/content/openie_allocations_aggregated.tsv', sep = '\t')
df_clausie = pd.read_csv('/content/clausie_allocations_aggregated.tsv', sep = '\t')
df_parse_0 = pd.read_csv('/content/parse_tree_clause_allocations.tsv', sep = '\t')

with open ('zero_alloc', 'rb') as fp:
    zero_alloc = pickle.load(fp)

In [ ]:
# Pre match commentary for the first match 
comment = df['Commentary'][0]

'Gus Poyet says the excitement at Sunderland has been reflected in the best week of training under his charge. The Capital One Cup final against Manchester City looms next weekend – Sunderland's first major final for 22 years – and there is also an FA Cup quarter-final at Brighton or Hull. Yet nothing matters more than Premier League survival and, with his team in the relegation zone, Poyet is desperate for a tonic. Arsenal want a response to the Champions League loss against Bayern Munich. David Hytner

In [ ]:
# Parse tree allocation
alloc = df_parse_0.groupby(['HomeTeam','AwayTeam','Date','Allocation']).apply(lambda x: x)

# Sunderland
sun = alloc.loc[ (alloc['Allocation'] == 'Sunderland') | (alloc['Allocation'] == 'Arsenal') ][0:6]['Sentence'][0:5]

# Arsenal
ars = alloc.loc[ (alloc['Allocation'] == 'Sunderland') | (alloc['Allocation'] == 'Arsenal') ][0:6]['Sentence'][6]

**Sunderland**

the excitement at sunderland has been reflected in the best week of training under his charge

gus poyet . says

the capital one cup final against manchester city there looms next weekend -- sunderland 's first major final for 22 years --

yet nothing with his team in the relegation zone , poyet matters more than premier league survival

yet nothing with his team in the relegation zone , poyet is desperate for a tonic

**Arsenal**

arsenal . want a response to the champions league loss against bayern munich

In [ ]:
# Aggregated home and away Sentences 
allocation_home = df_parse['Sentence'][0]
allocation_away = df_parse['Sentence'][1]


Home Team allocations

'the capital one cup final against manchester city there is also an fa cup quarter-final at brighton or hull arsenal . want a response to the champions league loss against bayern munich'

Away Team allocations

the excitement at sunderland has been reflected in the best week of training under his charge gus poyet . says the capital one cup final against manchester city there looms next weekend -- sunderland 's first major final for 22 years -- the capital one cup final against manchester city there is also an fa cup quarter-final at brighton or hull yet nothing with his team in the relegation zone , poyet matters more than premier league survival yet nothing with his team in the relegation zone , poyet is desperate for a tonic

0 : Draw

1 : Home Win

2 : Away Win

Dixon outputs

In [ ]:
dixon = np.array(df_dixon['PredResult']).reshape(-1,1)
print('Dixon and Cole prediction for match between Manchester City and Sunderland:',dixon[0])

Dixon and Cole prediction for match between Manchester City and Sunderland: [1]


Bookmakers outputs

In [ ]:
df['book_maker'] = np.where(
    df['OddsOutput'] == 'HW', 1, -1)


book = np.array(df['book_maker']).reshape(-1,1)

print('Bookmakers prediction for match between Manchester City and Sunderland:',book[0])

Bookmakers prediction for match between Manchester City and Sunderland: [1]


In [ ]:
# Match results
df['result'] = np.where(
    df['FTHG'] == df['FTAG'], 0, np.where(
    df['FTHG'] >  df['FTAG'], 1, -1))

Alpha values

In [ ]:
dfa = pd.read_csv('alpha_values.csv')
alpha_df = dfa.groupby(['Team']).mean().reset_index()
alpha_df.head()
dictn = dict(zip(alpha_df['Team'], alpha_df['Alpha']))
dictn
# Add a new column named 'alpha'
df['Alpha'] = df['HomeTeam'].map(dictn)
np.array(df['Alpha']).reshape(-1, 1).shape

print('Home Advantage factor for match between Manchester City(home) and Sunderland(away)',df['Alpha'][0])

Home Advantage factor for match between Manchester City(home) and Sunderland(away) 0.8280701754385966


Getting outputs from text data Using CountVectorizer and random forest

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

1) Splitting data into train and test. Data from last season is used as test. 


2) Vectorising the text using CountVectoriser. It creats a vocabulary of all the unique words in the corpus and counts the number of times a word appears in a document. 

In [ ]:
vectorizer = CountVectorizer(stop_words='english',binary=True,max_features=3000, ngram_range=(2,2))

def text(df):

  X_train, X_test = df.fillna('').astype(str)[:2042],df.fillna('').astype(str)[2042:]
  # X_train, X_test = df_openie['Sentence'].fillna('').astype(str)[:2042],df_openie['Sentence'].fillna('').astype(str)[2042:]
  # X_train, X_test = df_clausie['Sentence'].fillna('').astype(str)[:2042],df_clausie['Sentence'].fillna('').astype(str)[2042:]
  return X_train, X_test

X_train, X_test = text(df_parse['Sentence'])

xTr = vectorizer.fit_transform(X_train)
xTe = vectorizer.transform(X_test)


1) Splitting the vectorised data into home and away. Even entries belong to home teams and odd entries belong to away team. 

2) Applying home advantage to home team by taking the dot product of the vector alpha with the home vector.

3) Home and away vectors are concatenated. 

In [ ]:
hvtr = xTr.toarray()[::2]
avtr = xTr.toarray()[1::2]
hvte = xTe.toarray()[::2]
avte = xTe.toarray()[1::2]

hvtr = np.array(df['Alpha'][:1021]).reshape(-1, 1)*hvtr
hvte = np.array(df['Alpha'][1021:]).reshape(-1, 1)*hvte

xTr = np.concatenate((hvtr,avtr),axis = 1)
xTe = np.concatenate((hvte,avte),axis = 1)
print(xTr.shape,xTe.shape)


(1021, 6000) (277, 6000)


Training random forest on text data

In [ ]:
yTr = np.array(df['result'][:1021])
yTe = np.array(df['result'][1021:])

In [ ]:
model = RandomForestClassifier(n_estimators = 5, max_leaf_nodes= 25, random_state=2)

print(xTr.shape,xTe.shape)

model.fit(xTr, yTr)
print('Test accuracy for NLP Random Forest model:',model.score(xTe, yTe))

(1021, 6000) (277, 6000)
Test accuracy for NLP Random Forest model: 0.5126353790613718


In [ ]:
print('Training accuracy for NLP Random Forest model:',model.score(xTr, yTr))

Training accuracy for NLP Random Forest model: 0.5465230166503428


In [ ]:
# Comfusion matrix for NLP random forest showing how many samples from each class where predicted correctly
# and if predicted incorrectly what were they predicted as. 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(yTe, model.predict(xTe))
cm

array([[ 20,   2,  56],
       [  7,   0,  64],
       [  5,   1, 122]])

Uncomment the code for ensemble of just Dixon and bookmakers predictions 

In [ ]:
# Concatenating outputs from text model,dixon and bookmaker
yTr_predicted = model.predict(xTr).reshape(-1,1)
yTe_predicted = model.predict(xTe).reshape(-1,1)
Xr = np.concatenate((yTr_predicted,dixon[:1021].reshape(1021,-1),book[:1021]),axis =1)
Xe = np.concatenate((yTe_predicted,dixon[1021:].reshape(277,-1),book[1021:]),axis =1)

# Xr = np.concatenate((dixon[:1021].reshape(1021,-1),book[:1021]),axis =1)
# Xe = bnp.concatenate((dixon[:1021].reshape(1021,-1),book[:1021]),axis =1)

In [ ]:
# Training Ensemble 
E_model = RandomForestClassifier(n_estimators = 1,random_state=6)
E_model.fit(Xr, yTr)
print('Test accuracy for ensemble:',E_model.score(Xe, yTe))

Test accuracy for ensemble: 0.5848375451263538


In [ ]:
# Comfusion matrix for Ensemble showing how many samples from each class where predicted correctly
# and if predicted incorrectly what were they predicted as. 
cm = confusion_matrix(yTe, E_model.predict(Xe))
cm

array([[ 47,   5,  26],
       [ 18,  11,  42],
       [ 19,   5, 104]])

In [ ]:
print('training accuracy for ensemble',E_model.score(Xr, yTr))

training accuracy for ensemble 0.5925563173359452


Training and test accuracy of bookmakers

In [ ]:
tr_book = np.sum(book[:1021].reshape(1021) == yTr)/yTr.shape[0]
te_book = np.sum(book[1021:].reshape(277) == yTe)/yTe.shape[0]

print(f'Training accuracy for bookmakers is {tr_book}, and test accuaracy is {te_book}')

Training accuracy for bookmakers is 0.555337904015671, and test accuaracy is 0.5523465703971119
